In [52]:
import pandas as pd

In [53]:
data1=pd.read_csv("applnids_notin_tls203_appln_abstr.csv")

In [54]:
data2=pd.read_csv("patents_not_en_abstract.csv")

In [55]:
data = pd.concat([data1, data2], ignore_index=True)

In [56]:
data.head()

,key,appln_id
0,EP0147905A2,4379
1,EP0219694A2,4443
2,EP215657A2,4525
3,EP0214633A1,41453
4,EP0220904A2,57017


In [57]:
len(data)

17124

In [58]:
import mysql.connector
import os
from dotenv import load_dotenv

In [59]:
load_dotenv()
db_config = {
    'host': os.getenv('DB_HOST'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'database': os.getenv('DB_DATABASE')
}

In [60]:
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor(dictionary=True)
ids=tuple(data['appln_id'])
query = f"""
SELECT 
	t1.appln_id,   
    t1.docdb_family_id
FROM
	patstat_s24.tls201_appln t1
WHERE
    t1.appln_id IN {ids}
"""
cursor.execute(query)
results = cursor.fetchall()
cursor.close()

True

In [61]:
data3=pd.DataFrame(results)

In [62]:
data3.head()

,appln_id,docdb_family_id
0,667,3487358
1,673,8194882
2,713,27146044
3,4379,24256107
4,4390,8519350


In [63]:
len(data3)

17124

In [64]:
data4 = pd.merge(data, data3, on='appln_id', how='inner')

In [65]:
data4.head()

,key,appln_id,docdb_family_id
0,EP0147905A2,4379,24256107
1,EP0219694A2,4443,25116505
2,EP215657A2,4525,10585100
3,EP0214633A1,41453,27274544
4,EP0220904A2,57017,25148090


In [66]:
len(data4)

17124

In [67]:
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor(dictionary=True)
family_ids=tuple(data4['docdb_family_id'])
query = f"""
SELECT    
    t1.docdb_family_id,
    t1.appln_id,
    t1.appln_auth
FROM
	patstat_s24.tls201_appln t1
WHERE
    t1.docdb_family_id IN {family_ids}
"""
cursor.execute(query)
results = cursor.fetchall()
cursor.close()

True

In [68]:
data5=pd.DataFrame(results)

In [69]:
data5.head()

,docdb_family_id,appln_id,appln_auth
0,3487358,667,EP
1,8194882,673,EP
2,27146044,713,EP
3,24256107,4379,EP
4,8519350,4390,EP


In [70]:
len(data5)

118884

In [71]:
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor(dictionary=True)
ids=tuple(data5['appln_id'])
query = f"""
SELECT 
	t1.appln_id,   
    t1.appln_title
FROM
	patstat_s24.tls202_appln_title t1
WHERE
    t1.appln_id IN {ids}
    AND t1.appln_title_lg = 'en'
"""
cursor.execute(query)
results = cursor.fetchall()
cursor.close()

True

In [72]:
data6=pd.DataFrame(results)

In [73]:
len(data6)

76726

In [74]:
data6.head()

,appln_id,appln_title
0,667,"Rope formed of threads, yarns or twines made o..."
1,673,Indicating and function controlling optical un...
2,713,COMPOSITE PLATE WITH A LAYER OF NATURAL STONE
3,4379,"MULTI-COLOR MARKING IMPLEMENTS, AND METHOD AND..."
4,4390,METHOD TO MANUFACTURE COMPACTOR AND COMPACTOR ...


In [75]:
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor(dictionary=True)
ids=tuple(data6['appln_id'])
query = f"""
SELECT 
	t1.appln_id,   
    t1.appln_abstract
FROM
	patstat_s24.tls203_appln_abstr t1
WHERE
    t1.appln_id IN {ids}
    AND t1.appln_abstract_lg = 'en'
"""
cursor.execute(query)
results = cursor.fetchall()
cursor.close()

True

In [76]:
data7=pd.DataFrame(results)

In [77]:
data7.head()

,appln_id,appln_abstract
0,82688,The invention concerns a device for applying (...
1,97088,"The invention relates to a system, method and ..."
2,286537,"A construction for a shoe (20), particularly a..."
3,324253,There is provided a portable blower. The porta...
4,330532,"A method of producing a marking on diamond, pe..."


In [78]:
data8 = pd.merge(data6, data7, on='appln_id', how='inner')

In [79]:
data8.head()

,appln_id,appln_title,appln_abstract
0,82688,DEVICE FOR APPLYING A PRODUCT,The invention concerns a device for applying (...
1,97088,"SYSTEM, METHOD AND CAPSULE FOR PREPARING A BEV...","The invention relates to a system, method and ..."
2,286537,Shoe sole structures with enveloping side,"A construction for a shoe (20), particularly a..."
3,324253,Portable blower.,There is provided a portable blower. The porta...
4,330532,Method for producing a die insert,"A method of producing a marking on diamond, pe..."


In [80]:
len(data8)

35063

In [81]:
data9 = pd.merge(data5, data8, on='appln_id', how='inner')

In [82]:
data9.head()

,docdb_family_id,appln_id,appln_auth,appln_title,appln_abstract
0,8862541,82688,WO,DEVICE FOR APPLYING A PRODUCT,The invention concerns a device for applying (...
1,41328789,97088,WO,"SYSTEM, METHOD AND CAPSULE FOR PREPARING A BEV...","The invention relates to a system, method and ..."
2,23839637,286537,EP,Shoe sole structures with enveloping side,"A construction for a shoe (20), particularly a..."
3,27310129,324253,EP,Portable blower.,There is provided a portable blower. The porta...
4,27082389,330532,EP,Method for producing a die insert,"A method of producing a marking on diamond, pe..."


In [83]:
rank_order = ['EP', 'WO', 'US', 'CN', 'JP', 'KR', 'GB', 'CA', 'TW', 'DE', 'ES', 'RU', 'AU', 'FR', 'MX', 'UA', 'NZ']

In [84]:
rank_dict = {auth: rank for rank, auth in enumerate(rank_order)}

In [85]:
rank_dict

{'EP': 0,
 'WO': 1,
 'US': 2,
 'CN': 3,
 'JP': 4,
 'KR': 5,
 'GB': 6,
 'CA': 7,
 'TW': 8,
 'DE': 9,
 'ES': 10,
 'RU': 11,
 'AU': 12,
 'FR': 13,
 'MX': 14,
 'UA': 15,
 'NZ': 16}

In [86]:
data9['auth_rank'] = data9['appln_auth'].map(rank_dict)

In [87]:
data9.head()

,docdb_family_id,appln_id,appln_auth,appln_title,appln_abstract,auth_rank
0,8862541,82688,WO,DEVICE FOR APPLYING A PRODUCT,The invention concerns a device for applying (...,1.0
1,41328789,97088,WO,"SYSTEM, METHOD AND CAPSULE FOR PREPARING A BEV...","The invention relates to a system, method and ...",1.0
2,23839637,286537,EP,Shoe sole structures with enveloping side,"A construction for a shoe (20), particularly a...",0.0
3,27310129,324253,EP,Portable blower.,There is provided a portable blower. The porta...,0.0
4,27082389,330532,EP,Method for producing a die insert,"A method of producing a marking on diamond, pe...",0.0


In [89]:
data9 = data9.sort_values(['docdb_family_id', 'auth_rank'])

In [90]:
data9.head()

,docdb_family_id,appln_id,appln_auth,appln_title,appln_abstract,auth_rank
17303,574528,43515905,WO,BUILDING STRUCTURE,A building structure comprises interconnectabl...,1.0
24098,574528,48641948,US,Building structure,"PCT No. PCT/SE86/00457 Sec. 371 Date Jun. 7, 1...",2.0
8495,574528,15338498,DK,Building structure,A building structure comprises interconnectabl...,NaN
12201,574528,18480994,FI,Building construction - comprises assemblable ...,"The components (1,2,3) of the construction are...",NaN
20563,574586,47065159,WO,PROTECTIVE FACIAL MASK,A protective facial mask (10) covering the ora...,1.0


In [91]:
data10 = data9.drop_duplicates(subset='docdb_family_id', keep='first')

In [92]:
data10.head()

,docdb_family_id,appln_id,appln_auth,appln_title,appln_abstract,auth_rank
17303,574528,43515905,WO,BUILDING STRUCTURE,A building structure comprises interconnectabl...,1.0
20563,574586,47065159,WO,PROTECTIVE FACIAL MASK,A protective facial mask (10) covering the ora...,1.0
20653,574660,47067179,WO,STAIR CLIMBING EXERCISE APPARATUS,An exercise apparatus (10) that simulates stai...,1.0
7812,1226485,11643501,WO,PROCESS FOR BONDING HALOGENS TO SOLIDS IN THE ...,A process for bonding halogens to solids in th...,1.0
21287,1236211,47132595,WO,A METHOD FOR PRODUCING A MASS DISTRIBUTABLE PR...,A method for producing a mass distributable pr...,1.0


In [93]:
len(data10)

17092

In [94]:
data11 = data10.drop(columns=['appln_id'])

In [95]:
data11.head()

,docdb_family_id,appln_auth,appln_title,appln_abstract,auth_rank
17303,574528,WO,BUILDING STRUCTURE,A building structure comprises interconnectabl...,1.0
20563,574586,WO,PROTECTIVE FACIAL MASK,A protective facial mask (10) covering the ora...,1.0
20653,574660,WO,STAIR CLIMBING EXERCISE APPARATUS,An exercise apparatus (10) that simulates stai...,1.0
7812,1226485,WO,PROCESS FOR BONDING HALOGENS TO SOLIDS IN THE ...,A process for bonding halogens to solids in th...,1.0
21287,1236211,WO,A METHOD FOR PRODUCING A MASS DISTRIBUTABLE PR...,A method for producing a mass distributable pr...,1.0


In [96]:
len(data11)

17092

In [99]:
data12 = pd.merge(data4, data11, on='docdb_family_id', how='inner')

In [100]:
data12.head()

,key,appln_id,docdb_family_id,appln_auth,appln_title,appln_abstract,auth_rank
0,EP0147905A2,4379,24256107,US,Multi-color marking implement,This application discloses multi-color marking...,2.0
1,EP0219694A2,4443,25116505,US,Lock actuator assembly and card reader,An actuator assembly is mounted in a housing a...,2.0
2,EP215657A2,4525,10585100,US,Sonar transducers,A flextensional sonar transducer may comprise ...,2.0
3,EP0214633A1,41453,27274544,US,Control system for hydraulically-operated cons...,A control system for a hydraulic excavator hav...,2.0
4,EP0220904A2,57017,25148090,US,Article with laminated paper orientation for i...,The invention relates to an article comprising...,2.0


In [101]:
len(data12)

17092

In [102]:
len(data4)

17124

In [103]:
data13 = pd.merge(data4, data11, on='docdb_family_id', how='outer', indicator=True)

In [104]:
data14=data13[data13['_merge']!="both"]

In [105]:
data14.head()

,key,appln_id,docdb_family_id,appln_auth,appln_title,appln_abstract,auth_rank,_merge
12,EP0602033A1,16919016,1340219,NaN,NaN,NaN,NaN,left_only
13,EP0578632A1,16910918,1762664,NaN,NaN,NaN,NaN,left_only
413,EP1162330A2,15848760,3684009,NaN,NaN,NaN,NaN,left_only
1304,EP0206105A1,16623575,4234192,NaN,NaN,NaN,NaN,left_only
1699,EP1374961A2,15791480,5331371,NaN,NaN,NaN,NaN,left_only


In [106]:
len(data14)

32

In [107]:
17124-17092

32

In [109]:
data14[["key","appln_id","docdb_family_id"]].to_csv("patents_not_en.csv",index=False)

In [ ]:
data12.to_csv("13handling_non_english.csv",index=False)

In [ ]:
test=pd.read_csv("13handling_non_english.csv")

In [113]:
test.head()

,key,appln_id,docdb_family_id,appln_auth,appln_title,appln_abstract,auth_rank
0,EP0147905A2,4379,24256107,US,Multi-color marking implement,This application discloses multi-color marking...,2.0
1,EP0219694A2,4443,25116505,US,Lock actuator assembly and card reader,An actuator assembly is mounted in a housing a...,2.0
2,EP215657A2,4525,10585100,US,Sonar transducers,A flextensional sonar transducer may comprise ...,2.0
3,EP0214633A1,41453,27274544,US,Control system for hydraulically-operated cons...,A control system for a hydraulic excavator hav...,2.0
4,EP0220904A2,57017,25148090,US,Article with laminated paper orientation for i...,The invention relates to an article comprising...,2.0


In [114]:
len(test)

17092

In [118]:
test2=pd.read_csv("patents_not_en.csv")

In [119]:
test2.head()

,key,appln_id,docdb_family_id
0,EP0602033A1,16919016,1340219
1,EP0578632A1,16910918,1762664
2,EP1162330A2,15848760,3684009
3,EP0206105A1,16623575,4234192
4,EP1374961A2,15791480,5331371


In [120]:
len(test2)

32

In [ ]:
#patents with appln ids and family ids not containing english abstract and title = 32 (saved in patents_not_en.csv)